In [57]:
%load_ext autoreload
%autoreload 2

import os
import sys
from pathlib import Path

# Find the project root (the nearest ancestor that contains `src`) and add it to sys.path
cwd = Path.cwd()
project_root = cwd
for _ in range(6):
    if (project_root / "src").exists():
        break
    if project_root.parent == project_root:
        break
    project_root = project_root.parent
else:
    project_root = cwd

proj_path = str(project_root.resolve())
if proj_path not in sys.path:
    # Insert at front so local packages take precedence
    sys.path.insert(0, proj_path)

import aerosandbox.numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from src.airfoil.compute_airfoil_quality import compute_airfoil_quality, QualityError
from aerosandbox import Airfoil
from src.airfoil import airfoil_modifications

import plotly.express as px

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Extraction

In [3]:
airfoil_database_path = asb._asb_root / "geometry" / "airfoil" / "airfoil_database"

airfoil_database = [
    asb.Airfoil(name=filename.stem).normalize()
    for filename in airfoil_database_path.glob("*.dat")
]

## Preprocessing

### Quality check

In [ ]:
# Remove airfoils that don't pass the quality tests
quality_airfoil_database = airfoil_database.copy()

for af in airfoil_database:
    try:
        compute_airfoil_quality(af, airfoil_database_path)
    except QualityError as e:
        quality_airfoil_database.remove(af)
        print(f"Airfoil {af.name.ljust(20)} failed quality checks: {e}")
        # af.draw()

Airfoil as6095               failed quality checks: Airfoil has negative thickness.
Airfoil fx79w470a            failed quality checks: Airfoil has abnormally large changes in angle at (0.977417, 0.0876827), 21.6 deg.
Airfoil fx79w660a            failed quality checks: Airfoil has abnormally large changes in angle at (0.948442, -0.203101), 34.7 deg.
Airfoil mh112                failed quality checks: Airfoil has abnormally high x-coordinates.


In [78]:
from aerosandbox.geometry.airfoil.airfoil_families import get_kulfan_parameters

LW_CAP = 8.0
UW_CAP = 8.0
THICKNESS_CAP = 0.01
LE_CAP = 2

def build_dataset(airfoils_database: list):
  airfoils_data = []

  for airfoil in airfoils_database:
    try:
      parameters = get_kulfan_parameters(
          coordinates=airfoil.coordinates,
          n_weights_per_side=12,
          normalize_coordinates=False,
          use_leading_edge_modification=False,
      )

      # Remove outliers
      _lw = parameters["lower_weights"]
      _uw = parameters["upper_weights"]
      _te = parameters["TE_thickness"]
      _le = parameters["leading_edge_weight"]
      if any(_lw > LW_CAP) or any(_uw > UW_CAP) or (_te > THICKNESS_CAP) or (-1 * LE_CAP > _le or _le > LE_CAP):
        continue

      airfoils_data.append([airfoil.name, airfoil.coordinates, parameters])
    except:
      # airfoils_data.append([airfoil.name, airfoil.coordinates, None])
      pass

  airfoil_dataset = pd.DataFrame(airfoils_data, columns=["airfoil_name", "coordinates", "kulfan_parameters"])
  airfoil_dataset["shape"] = airfoil_dataset.coordinates.apply(lambda coords: coords.shape)
  airfoil_dataset["points"] = airfoil_dataset["shape"].apply(lambda shape: shape[0])

  return airfoil_dataset

airfoil_dataset = build_dataset(quality_airfoil_database)
display(airfoil_dataset)

,airfoil_name,coordinates,kulfan_parameters,shape,points
0,2032c,"[[1.0, 0.0016], [0.95, 0.0124], [0.9, 0.0229],...","{'lower_weights': [-0.10683679112985149, -0.14...","(35, 2)",35
1,a18,"[[0.9999521810074102, 0.0030692549800954484], ...","{'lower_weights': [-0.0748249423080259, 0.1111...","(41, 2)",41
2,a18sm,"[[1.000000015325, 0.0030649999999233747], [0.9...","{'lower_weights': [-0.1149933895082661, -0.083...","(79, 2)",79
3,a63a108c,"[[1.0000122498499395, 0.0034999571255252136], ...","{'lower_weights': [-0.1416319727825609, 0.1800...","(99, 2)",99
4,ag03,"[[1.0000001216057246, 0.000605001790562621], [...","{'lower_weights': [-0.06783052052881457, -0.09...","(180, 2)",180
...,...,...,...,...,...
2019,Zone-25,"[[1.0000002713693834, 0.0003249965234413304], ...","{'lower_weights': [-0.08496360007168066, -0.05...","(257, 2)",257
2020,Zone-36,"[[1.00000015242192, 0.0003349965806822195], [0...","{'lower_weights': [-0.09319425931547737, -0.09...","(257, 2)",257
2021,Zone-40,"[[1.000000062998738, 0.0003499964886953402], [...","{'lower_weights': [-0.10342948998864263, -0.13...","(257, 2)",257
2022,Zone-46,"[[1.000000045624087, 0.0003649963443335458], [...","{'lower_weights': [-0.10430080888311542, -0.14...","(257, 2)",257


### Second Quality Check


In [79]:
airfoil_dataset["lower_weights"] = airfoil_dataset["kulfan_parameters"].apply(lambda params: params["lower_weights"])
airfoil_dataset["upper_weights"] = airfoil_dataset["kulfan_parameters"].apply(lambda params: params["upper_weights"])
airfoil_dataset["TE_thickness"] = airfoil_dataset["kulfan_parameters"].apply(lambda params: params["TE_thickness"])
airfoil_dataset["leading_edge_weight"] = airfoil_dataset["kulfan_parameters"].apply(lambda params: params["leading_edge_weight"])

lw = airfoil_dataset["lower_weights"].to_numpy()
lw = np.vstack(lw)
lw = lw.flatten()

le = airfoil_dataset["leading_edge_weight"].to_numpy()
le = np.vstack(le)
le = le.flatten()

te = airfoil_dataset["TE_thickness"].to_numpy()
te = np.vstack(te)
te = te.flatten()

# Print a boxplot to visualize the distribution of lower_weights
fig = px.box(
    pd.DataFrame(lw),
    points="all",
    title="Distribution of Lower Weights",
)
fig.show()

fig = px.box(
    pd.DataFrame(le),
    points="all",
    title="Distribution of Leading Edge Weights",
)
fig.show()

fig = px.box(
    pd.DataFrame(te),
    points="all",
    title="Distribution of TE Thickness",
)
fig.show()

In [88]:
a = airfoil_dataset.sort_values(by="leading_edge_weight", inplace=False, ascending=True)
print(a["leading_edge_weight"].iloc[0])
Airfoil(coordinates=a["coordinates"].iloc[0]).draw()

-1.9724272403874044


In [44]:
fig = px.histogram(airfoil_dataset, x="points", title="Total number of points distribution")
fig.show()

### Coordinates standardization

In [82]:
# The number of coordinates for each airfoil is inconsistent across the database, so we use Cubic splines interpolation to standadize the coordinates
n_points_per_side = 75
normalized_coords = [Airfoil(coordinates=coords).repanel(n_points_per_side).coordinates for coords in airfoil_dataset["coordinates"]]

std_airfoil_dataset = airfoil_dataset.copy()
std_airfoil_dataset["coordinates"] = normalized_coords
std_airfoil_dataset["shape"] = std_airfoil_dataset.coordinates.apply(lambda coords: coords.shape)
std_airfoil_dataset["points"] = std_airfoil_dataset["shape"].apply(lambda shape: shape[0])
display(std_airfoil_dataset)

,airfoil_name,coordinates,kulfan_parameters,shape,points,lower_weights,upper_weights,TE_thickness,leading_edge_weight
0,2032c,"[[1.0, 0.0016], [0.9995417966629326, 0.0016995...","{'lower_weights': [-0.10683679112985149, -0.14...","(149, 2)",149,"[-0.10683679112985149, -0.14856251074990626, 0...","[0.0948583634292328, 0.2110741405929873, 0.381...",0.003200,0.656100
1,a18,"[[0.9999521810074102, 0.0030692549800954484], ...","{'lower_weights': [-0.0748249423080259, 0.1111...","(149, 2)",149,"[-0.0748249423080259, 0.11111203561026745, -0....","[0.14708768903492125, 0.374267031945715, -0.10...",0.006124,-0.451487
2,a18sm,"[[1.000000015325, 0.0030649999999233747], [0.9...","{'lower_weights': [-0.1149933895082661, -0.083...","(149, 2)",149,"[-0.1149933895082661, -0.08387653449556368, -0...","[0.11607345762901729, 0.1398997752966345, 0.19...",0.006122,0.124729
3,a63a108c,"[[1.0000122498499395, 0.0034999571255252136], ...","{'lower_weights': [-0.1416319727825609, 0.1800...","(149, 2)",149,"[-0.1416319727825609, 0.18009771209253028, -0....","[0.2166156880331907, 0.02823720105628863, 0.50...",0.007224,-0.226960
4,ag03,"[[1.0000001216057246, 0.000605001790562621], [...","{'lower_weights': [-0.06783052052881457, -0.09...","(149, 2)",149,"[-0.06783052052881457, -0.09985633030632891, 0...","[0.12321441485680125, 0.15327355589418362, 0.0...",0.001176,0.096844
...,...,...,...,...,...,...,...,...,...
2019,Zone-25,"[[1.0000002713693834, 0.0003249965234413304], ...","{'lower_weights': [-0.08496360007168066, -0.05...","(149, 2)",149,"[-0.08496360007168066, -0.054415346990110074, ...","[0.1117065421950243, 0.11175453585261387, 0.08...",0.000628,0.105904
2020,Zone-36,"[[1.00000015242192, 0.0003349965806822195], [0...","{'lower_weights': [-0.09319425931547737, -0.09...","(149, 2)",149,"[-0.09319425931547737, -0.0950206820874855, 0....","[0.10116500328671965, 0.0605159760595101, 0.19...",0.000632,0.220925
2021,Zone-40,"[[1.000000062998738, 0.0003499964886953402], [...","{'lower_weights': [-0.10342948998864263, -0.13...","(149, 2)",149,"[-0.10342948998864263, -0.13742488229405458, 0...","[0.09189924459864875, 0.02399273136743037, 0.2...",0.000656,0.337661
2022,Zone-46,"[[1.000000045624087, 0.0003649963443335458], [...","{'lower_weights': [-0.10430080888311542, -0.14...","(149, 2)",149,"[-0.10430080888311542, -0.1463960461075766, 0....","[0.09226736500903589, 0.02493185988106343, 0.2...",0.000682,0.353729


In [83]:
std_airfoil_dataset["TE_thickness"] = std_airfoil_dataset["kulfan_parameters"].apply(lambda params: params["TE_thickness"])
std_airfoil_dataset["leading_edge_weight"] = std_airfoil_dataset["kulfan_parameters"].apply(lambda params: params["leading_edge_weight"])
std_airfoil_dataset[["TE_thickness", "leading_edge_weight"]].describe()

,TE_thickness,leading_edge_weight
count,2024.000000,2024.000000
mean,0.001133,0.152098
std,0.001984,0.521721
min,0.000000,-1.972427
25%,0.000000,-0.008786
50%,0.000002,0.146345
75%,0.001577,0.374835
max,0.009997,1.990688


In [84]:
std_airfoil_dataset["points"].describe()

count    2024.0
mean      149.0
std         0.0
min       149.0
25%       149.0
50%       149.0
75%       149.0
max       149.0
Name: points, dtype: float64

### Saving dataset

In [85]:
# std_airfoil_dataset.to_pickle(rf"../data/processed/airfoil_dataset_{n_points_per_side}.pkl")
std_airfoil_dataset.to_json(rf"../../data/processed/kulfan_dataset_{n_points_per_side}.json")